# ML Căn bản 


## 4.1 4 nhánh chính của ML

Cả 3 ví dụ trước đều là **supervised learning**  : Học mối quan hệ từ input với output. 
Tuy vậy ML vẫn còn rất nhiều dạng khác nhau 

### Supervised Learning
- OCR, speech recognition, image classification, language translation

### Unsupervised Learning
- Data visualization, data compression, denoising ( analytics )
- ALgorithm :Clustering, Demensionality reduction, 

### Self-supervised Learning 
E.g : Auto encoder

### Reinforcement learning
- Keywords: agent, action, reward, score
E.g : Google deepmind's Alpha Go, OpenAI's dota2 SF bot ,Travis :D

## 4.2 Thuật ngữ 

- sample / input 
- prediction / output
- target : the truth
- prediction error / loss value: distance between prediction and target
- Classes : set of possible labels
- Label : instance of a class annotation
- Ground-truth/ annotation : targets for a data set ( typically collected by humans )
- Binary classification
- Multi-class classification
- Multi-label classification : each input can be assigned multiple labels ( 1 images with both dog and cat -> dog, cat )
- Scalar regression : target is a continuous scalar value (e.g: House price)
- Vector regression : target is a set of continuous values ( e.g: coordinates of bounding boxes )

## 4.3 Đánh giá models

Một model tốt là 1 model có lời giải **tổng quát nhất** ( hoạt động tốt cho data ngoài tập training ), và over-fitting là vấn đề cốt lõi mà các model gặp phải .

--> Chia data thành các tập training, validation và test 

### 4.3.1 Tập training, validation và test 

Đánh giá 1 model hiểu đơn giản là chia data thành 3 phần : training, validation và test. 
- tập training và tập validation : Tùy chỉnh parameter cho model
  - tập training : sử dụng để train model
  - tập validation : đánh giá tạm thời kết quả của model vừa train
  
- tập test : Sau khi có 1 model ổn nhất, ta sẽ đánh giá model đó bằng tập test

Tại sao không đơn giản chỉ chia 2 tập là training và test ???
  > Sẽ dẫn đến over fitting đối với test set và model không tổng quát

**Over fitting test set** là gì ?
  > Khái niệm này gọi là "Information leak", khi model của bạn bắt đầu học kết quả từ data của test set 
  
Model **không hề train với test set**, tại sao lại **có thể học được thông tin** trong data đó ?
  > Sau khi train 1 model, bạn sẽ dùng test set để đánh giá độ chính xác của model --> Sau đó model được "tuning" : thay đổi mô hình, số layers, số unit mỗi layer, optimizer và loss function ... để kết quả cải thiện hơn --> Quá trình tuning cũng là quá trình học, và lặp đi lặp lại quá trình (train, tuning) bằng cách đánh giá sai số qua test set dần dần sẽ làm model nhìn được toàn bộ test set --> over fitting 
  
Nhưng nếu **test set chỉ sử dụng 1 lần** thì vẫn có thể đánh giá độ chính xác của model đúng không ? 
  > Đúng vậy, đó là nguyên nhân ta chia data thành 3 tập : training, validation và test, và tập test cách biệt hoàn toàn với 2 tập còn lại

Dưới đấy là một số phương pháp validation cơ bản 

### 4.3.2 Hold-out validation

```python
num_validation_samples = 10000
num_test_samples = 10000
# shuffling the data is usually appropriate
np.random.shuffle(data)
# splitting data into training, validation & test
validation_data	= data[:num_validation_samples]
data = [num_validation_samples:]
test_data = data[:num_test_samples]
data = [num_test_samples:]
training_data = data[:]
# get_model return a model that you've built
model = get_model()
# traing
model.train(training_data)
validation_score = model.evaluate(training_data)
# at this point you can tune your model,
# retrain it, evaluate it, tune it again...
# once you have tuned your hyperparameters,
# is it common to train your final model from scratch
# on all non-test data available.
# note that the "+" operator here is list concatenation,
# not summation.
model = get_model()
test_score = model.evaluate(training_data + validation_data)
```

Đây là phương pháp đơn giản nhất, nhưng có **1 điểm yếu** : Khi **lượng data quá nhỏ** --> **val và test set chứa quá ít samples** để đánh giá tổng quát được model. Rất dễ nhận thấy khi ta shuffle data trước khi split, độ chính xác của model bị thay đổi rất nhiều --> **K-fold validation hoặc iterated K-fold validation** là 2 pp hiệu quả hơn

### 4.3.3 K-fold validation

Chia data thành N phần bằng nhau, lấy 1 phần làm val và train model trên N-1 phần còn lại, lấy sai số , lặp lại (fold) N lần . Kết quả cuối cùng là avg của các sai số 

#### Shuffle data :
```python
seed = np.arange(train_data.shape[0])
# trong vong lap 
np.random.shuffle(s)
train_data = train_data[s]
train_targets = train_targets[s]
```

### 4.3.4 Iterated K-fold validation with shuffling

Đánh giá model khá tốt khi bạn có ít data. Ta lặp lại K-fold nhiều lần, và trước mỗi lần đều shuffle data trước khi split ra K phần . 

*Lưu ý* : ta train và evaluate **P*K models**, với **P là số vòng lặp**

### 4.3.5 Vài lời cuối 

Bạn nên lưu ý 1 vài điều sau :
- **Data representativity** : Trong bài toán classify digit, bạn chia data không đều, tập training chỉ có các số từ 0-7, tập test toàn sample của 8 và 9 --> Một lỗi rất dễ mắc phải, hãy shuffle random phù hợp 
- **The arrow of time** : Bạn cần **predict tương lai** dựa trên data quá khứ . **Không nên shuffle data**, hãy đảm bảo rằng tất cả test data đều là tương lai của train data
- **Redundancy** : Sự trùng lặp data, khi data có **một số sample bị lặp**, sau khi chia data, train và test set có thể tồn tại 1 số sample trùng nhau. Hãy đảm bảo rằng data trong tập train và test **luôn tách biệt**